# Postprocessing output files

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
chemblpath = 'chembl_35_sqlite/chembl_35.db' #path must be modified accordingly

### I. Creating output file for the NER method detection and BAO linking

### 1. Merging NER output with BAO output

In [2]:
# Loads output files from NER method extraction and BAO linking
ner_data = pd.read_csv("1_NER_method/Results/ner_chembl_35.tsv", sep='\t')
bao_data = pd.read_csv("3_entity_linking_BAO/results/chembl35_assays_bao_annotations_0_6.tsv", sep='\t')
ner_data

,assay_id,year,description,assay_type,method
0,89,1994.0,Evaluated for its activity to inhibit rat live...,B,NaN
1,128,2002.0,Tested in vitro for inhibition of chymotrypsin...,B,NaN
2,248,1982.0,"Percent inhibition was measured against 5,10-M...",B,NaN
3,235,2003.0,Inhibitory activity against inosine 5'-inosine...,B,NaN
4,336,1989.0,Binding affinity to the receptor was determine...,F,NaN
...,...,...,...,...,...
1169288,2339315,2023.0,Antiviral activity against wild type Vaccinia ...,F,microscopic analysis
1169289,2327448,2024.0,Binding affinity to Streptomyces peucetius 4-O...,B,NaN
1169290,2348461,2024.0,Induction of apoptosis in human HCCLM3 cells a...,F,Annexin V/FITC-PI staining based flow cytometry
1169291,2370617,2023.0,Induction of apoptosis in human MV4-11 cells a...,F,Annexin V-FITC staining based flow cytometry


In [3]:
# shows numbers
display(ner_data.head(1))
display(bao_data.head(1))

print(len(ner_data))
print(len(bao_data))

,assay_id,year,description,assay_type,method
0,89,1994.0,Evaluated for its activity to inhibit rat live...,B,NaN


,assay_id,year,description,assay_type,method,split_method,Source Term ID,Source Term,Mapped Term Label,Mapped Term CURIE,Mapped Term IRI,Mapping Score
0,949,2003.0,In vitro binding affinity by radioligand bindi...,B,In vitro binding affinity | radioligand bindin...,In vitro binding affinity,http://ccb.hms.harvard.edu/t2t/RttK3rWfuXh,In vitro binding affinity,binding,GO:0005488,http://purl.obolibrary.org/obo/GO_0005488,0.6


1169293
155426


In [4]:
# Renaming columns
bao_data = bao_data.rename(columns={
    'Source Term ID': 'source_term_id',
    'Source Term': 'source_term',
    'Mapped Term Label': 'mapped_term_label',
    'Mapped Term CURIE': 'mapped_term_curie',
    'Mapped Term IRI': 'mapped_term_iri',
    'Mapping Score': 'mapping_score'
})


In [5]:
# First, define the columns to keep from ner_data
merge_keys = ['assay_id', 'year', 'description', 'assay_type']

In [6]:
# Columns from bao_data that need to be aggregated
# This assumes all columns after 'method' in bao_data, excluding 'split_method',
# need to be aggregated. Adjust if there are other columns.
aggregation_columns = [col for col in bao_data.columns if col not in merge_keys + ['method','Tags']]

In [7]:
# Function to aggregate unique values into a comma-separated string
def aggregate_unique(series):
    # Convert all elements to string, handle NaNs, and then join unique values
    # Use str(x) to handle potential non-string types, then filter out 'nan' string
    unique_values = sorted([str(x) for x in series.dropna().unique() if pd.notna(x)])
    if not unique_values:
        return np.nan # Return NaN if no valid unique values are found
    return ','.join(unique_values)

In [8]:
# --- Step 1: Pre-process bao_data by grouping and aggregating ---
# For each group, apply the aggregate_unique function to specified columns
# These are the columns you explicitly want to group by.
bao_grouping_keys = merge_keys + ['method', 'split_method']

# Define the columns that should be AGGREGATED.
# These are all columns in bao_data that are *not* part of the grouping keys.
aggregation_columns = [col for col in bao_data.columns if col not in bao_grouping_keys]
aggregated_bao_data = bao_data.groupby(bao_grouping_keys).agg(
    {col: aggregate_unique for col in aggregation_columns}
).reset_index()

In [9]:
# --- Step 2: Explode the 'method' column in ner_data ---
ner_data_exploded = ner_data.copy()
# Conditionally split or assign np.nan
def split_method_conditionally(method_value):
    if pd.isna(method_value): # If it's already np.nan or None
        return [np.nan] # Return a list containing np.nan for explode
    return [s.strip() for s in str(method_value).split('|')] # Otherwise, convert to string and split

ner_data_exploded['method_list'] = ner_data_exploded['method'].apply(split_method_conditionally)
ner_data_exploded = ner_data_exploded.explode('method_list')
ner_data_exploded = ner_data_exploded.rename(columns={'method_list': 'method_single'})

In [10]:
# --- Step 3: Merge ner_data_exploded with aggregated_bao_data ---
# Aligning column names for merge
temp_aggregated_bao_data = aggregated_bao_data.rename(columns={'method': 'method_from_bao'})

# Perform the left merge
# keep all individual methods from ner_data_exploded.
# match 'method_single' from ner_data_exploded with 'split_method' from aggregated_bao_data.
# The common keys (excluding methods) are used directly.
merged_df_with_unmatched = pd.merge(
    ner_data_exploded,
    temp_aggregated_bao_data,
    left_on=merge_keys + ['method_single'],
    right_on=merge_keys + ['split_method'], # Match 'method_single' with 'split_method'
    how='left'
)

In [11]:
# Clean up the dataframe by dropping redundant method columns and reordering columns.
merged_df_final = merged_df_with_unmatched.drop(columns=['method_from_bao'], errors='ignore')
merged_df_final = merged_df_final.rename(columns={'method_single': 'individual_method'})

### 2. Adding extra columns

In [12]:
display(merged_df_final.head(1))
display(len(merged_df_final))

,assay_id,year,description,assay_type,method,individual_method,split_method,source_term_id,source_term,mapped_term_label,mapped_term_curie,mapped_term_iri,mapping_score
0,89,1994.0,Evaluated for its activity to inhibit rat live...,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1188094

In [13]:
# Get all unique assay_id values from DataFrame
# Convert to a list or tuple for the IN clause
assay_ids_to_query = tuple(merged_df_final['assay_id'].tolist())

sql_query = f"""
SELECT
    a.assay_id,
    a.chembl_id as assay_chembl_id,
    d.chembl_id as document_chembl_id,
    a.bao_format,
    b.label as bao_label,
    d.pubmed_id,
    d.chembl_release_id
FROM
    assays a
JOIN
    docs d ON a.doc_id = d.doc_id
JOIN
    bioassay_ontology b ON a.bao_format = b.bao_id"""

In [14]:
# Connect to DB
connection = sqlite3.connect(chemblpath)

In [15]:
# Execute the Query and Fetch Results into a DataFrame
results_df = pd.read_sql(sql_query, connection)

In [16]:
results_df.head()

,assay_id,assay_chembl_id,document_chembl_id,bao_format,bao_label,pubmed_id,chembl_release_id
0,56785,CHEMBL667932,CHEMBL1139451,BAO_0000218,organism-based format,14695813.0,1
1,81515,CHEMBL692414,CHEMBL1139451,BAO_0000218,organism-based format,14695813.0,1
2,81517,CHEMBL692416,CHEMBL1139451,BAO_0000218,organism-based format,14695813.0,1
3,81523,CHEMBL693586,CHEMBL1139451,BAO_0000218,organism-based format,14695813.0,1
4,81525,CHEMBL693588,CHEMBL1139451,BAO_0000218,organism-based format,14695813.0,1


In [17]:
len(results_df)

1740546

In [18]:
results_df['assay_id'].nunique()

1740546

In [19]:
results_assay_ids = results_df.loc[results_df['assay_id'].isin(assay_ids_to_query)]

In [20]:
len(results_assay_ids)

1169293

In [21]:
# Merge results into one dataframe
dataset = pd.merge(merged_df_final, results_assay_ids, on='assay_id', how='left')

### 3. Cleaning or modifying main dataset after creation

In [22]:
# Reads the main dataset
#dataset = pd.read_csv("main_ner_bao_chembl_35.tsv", sep='\t')


In [28]:
# Clean the main dataset columns and subset specific columns
dataset['year'] = dataset['year'].astype('Int64') #it corrects the year format
dataset['pubmed_id'] = pd.to_numeric(dataset['pubmed_id'], errors='coerce').astype('Int64').astype(str) #it corrects the pubmed to string
dataset = dataset[['assay_id', 'assay_chembl_id', 'description', 'assay_type', 'method', 'individual_method', 'mapped_term_label', 'mapped_term_curie', 'mapped_term_iri', 'mapping_score','year', 'document_chembl_id', 'bao_format', 'bao_label', 'pubmed_id', 'chembl_release_id']]
dataset.to_csv("main_ner_bao_chembl_35.tsv", sep='\t', index=False)

In [29]:
# sample_data
subset = dataset.head(100)
subset.to_csv("main_ner_bao_chembl_35_sample.tsv", sep='\t', index=False)
subset.head(10)

,assay_id,assay_chembl_id,description,assay_type,method,individual_method,mapped_term_label,mapped_term_curie,mapped_term_iri,mapping_score,year,document_chembl_id,bao_format,bao_label,pubmed_id,chembl_release_id
0,89,CHEMBL615209,Evaluated for its activity to inhibit rat live...,B,NaN,NaN,NaN,NaN,NaN,NaN,1994,CHEMBL1127895,BAO_0000019,assay format,<NA>,1
1,128,CHEMBL615352,Tested in vitro for inhibition of chymotrypsin...,B,NaN,NaN,NaN,NaN,NaN,NaN,2002,CHEMBL1136019,BAO_0000220,subcellular format,11992770,1
2,248,CHEMBL615109,"Percent inhibition was measured against 5,10-M...",B,NaN,NaN,NaN,NaN,NaN,NaN,1982,CHEMBL1122152,BAO_0000357,single protein format,7057422,1
3,235,CHEMBL615267,Inhibitory activity against inosine 5'-inosine...,B,NaN,NaN,NaN,NaN,NaN,NaN,2003,CHEMBL1136691,BAO_0000357,single protein format,12657279,1
4,336,CHEMBL615309,Binding affinity to the receptor was determine...,F,NaN,NaN,NaN,NaN,NaN,NaN,1989,CHEMBL1124386,BAO_0000218,organism-based format,2913295,1
5,512,CHEMBL615535,compound was evaluated for their acute effects...,F,NaN,NaN,NaN,NaN,NaN,NaN,1996,CHEMBL1129417,BAO_0000218,organism-based format,8941393,1
6,513,CHEMBL615536,compound was evaluated for their acute effects...,F,NaN,NaN,NaN,NaN,NaN,NaN,1996,CHEMBL1129417,BAO_0000218,organism-based format,8941393,1
7,35,CHEMBL615147,Compound at 100 uM was tested in vitro for inh...,B,NaN,NaN,NaN,NaN,NaN,NaN,1996,CHEMBL1129426,BAO_0000357,single protein format,8831760,1
8,351,CHEMBL615407,Inhibition constant for binding to Zn2+ form o...,B,NaN,NaN,NaN,NaN,NaN,NaN,2001,CHEMBL1134091,BAO_0000357,single protein format,11412967,1
9,437,CHEMBL615597,Inhibition of 6 nM bombesin induced [3H]thymid...,F,NaN,NaN,NaN,NaN,NaN,NaN,1994,CHEMBL1127325,BAO_0000219,cell-based format,8120863,1


### II. Creating output file with metadata columns for the broad assay category prediction results

#### 1. Loading main dataset and merging with ChEMBL metadata

In [47]:
broad_cat_data = pd.read_csv("2_broad_assay_category/chembl_35_broad_results_processed.txt", sep='\t')

In [48]:
broad_cat_data['assay_id'].nunique()

1169293

In [49]:
results_assay_ids['assay_id'].nunique()

1169293

In [50]:
# Merge broad assay category predictions with ChEMBL metadata
broad_merged = pd.merge(broad_cat_data, results_assay_ids, on='assay_id')

In [51]:
broad_merged['assay_id'].nunique()

1169293

In [52]:
broad_merged['year'] = broad_merged['year'].astype('Int64') #it corrects the year format
broad_merged['pubmed_id'] = pd.to_numeric(broad_merged['pubmed_id'], errors='coerce').astype('Int64').astype(str) #it corrects the pubmed to string
broad_merged['prediction_score'] = broad_merged['prediction_score'].apply(lambda x: f'{x:.3f}')
broad_merged['Radioligand binding (BAO_0002776)'] = broad_merged['Radioligand binding (BAO_0002776)'].apply(lambda x: f'{x:.3f}')
broad_merged['Binding (BAO_0002989)'] = broad_merged['Binding (BAO_0002989)'].apply(lambda x: f'{x:.3f}')
broad_merged['Protein activity (BAO_0013016)'] = broad_merged['Protein activity (BAO_0013016)'].apply(lambda x: f'{x:.3f}')
broad_merged['in vivo method (BAO_0040021)'] = broad_merged['in vivo method (BAO_0040021)'].apply(lambda x: f'{x:.3f}')
broad_merged['Cell phenotype (BAO_0002542)'] = broad_merged['Cell phenotype (BAO_0002542)'].apply(lambda x: f'{x:.3f}')
broad_merged['Nucleic acid binding'] = broad_merged['Nucleic acid binding'].apply(lambda x: f'{x:.3f}')
broad_merged['Antimicrobial activity'] = broad_merged['Antimicrobial activity'].apply(lambda x: f'{x:.3f}')

In [53]:
broad_merged.head()

,assay_id,year,description,assay_type,predicted_category,prediction_score,predicted_bao_id,predicted_bao_term,Radioligand binding (BAO_0002776),Binding (BAO_0002989),...,in vivo method (BAO_0040021),Cell phenotype (BAO_0002542),Nucleic acid binding,Antimicrobial activity,assay_chembl_id,document_chembl_id,bao_format,bao_label,pubmed_id,chembl_release_id
0,89,1994,Evaluated for its activity to inhibit rat live...,B,Protein activity,0.821,BAO_0013016,functional target-based,0.000,0.021,...,0.000,0.000,0.000,0.004,CHEMBL615209,CHEMBL1127895,BAO_0000019,assay format,<NA>,1
1,235,2003,Inhibitory activity against inosine 5'-inosine...,B,Protein activity,0.760,BAO_0013016,functional target-based,0.000,0.000,...,0.000,0.369,0.000,0.003,CHEMBL615267,CHEMBL1136691,BAO_0000357,single protein format,12657279,1
2,667,2003,5-hydroxytryptamine 1A receptor antagonism was...,F,Radioligand binding,0.749,BAO_0002776,radioligand binding assay,0.749,0.004,...,0.000,0.005,0.005,0.000,CHEMBL615743,CHEMBL1136114,BAO_0000019,assay format,12824042,1
3,758,2001,Binding affinity towards human 5-hydroxytrypta...,B,Binding,1.000,BAO_0002989,binding assay,0.001,1.000,...,0.000,0.000,0.001,0.000,CHEMBL615860,CHEMBL1134342,BAO_0000357,single protein format,11170639,1
4,1155,1992,Compound was evaluated for in vitro binding af...,B,Radioligand binding,1.000,BAO_0002776,radioligand binding assay,1.000,0.000,...,0.001,0.000,0.000,0.001,CHEMBL616100,CHEMBL1126190,BAO_0000221,tissue-based format,1353116,1


#### 2. Renaming and reorganizing columns

In [54]:
# Rename columns
broad_merged.rename(columns={'Radioligand binding (BAO_0002776)': 'Radioligand binding (BAO_0002776) prediction score'
                , 'Binding (BAO_0002989)': 'Binding (BAO_0002989) prediction score'
                , 'Protein activity (BAO_0013016)': 'Protein activity (BAO_0013016) prediction score'
                , 'in vivo method (BAO_0040021)': 'in vivo method (BAO_0040021) prediction score'
                , 'Cell phenotype (BAO_0002542)': 'Cell phenotype (BAO_0002542) prediction score'
                , 'Nucleic acid binding': 'Nucleic acid binding prediction score'
                , 'Antimicrobial activity': 'Antimicrobial activity prediction score'
               }, inplace=True)

In [55]:
# Decide on column order
broad_merged = broad_merged[['assay_id', 'assay_chembl_id', 'description', 'assay_type', 'predicted_category',
       'prediction_score', 'predicted_bao_id', 'predicted_bao_term',
       'Radioligand binding (BAO_0002776) prediction score',
       'Binding (BAO_0002989) prediction score',
       'Protein activity (BAO_0013016) prediction score',
       'in vivo method (BAO_0040021) prediction score',
       'Cell phenotype (BAO_0002542) prediction score',
       'Nucleic acid binding prediction score',
       'Antimicrobial activity prediction score', 'year', 'document_chembl_id', 'bao_format', 'bao_label', 'pubmed_id', 'chembl_release_id']]

#### 3. Saving file

In [56]:
broad_merged.to_csv("main_broad_category_chembl_35.tsv", sep='\t', index=False)

In [57]:
# sample_data
subset = broad_merged.head(100)
subset.to_csv("main_broad_category_chembl_35_sample.tsv", sep='\t', index=False)
subset.head(10)

,assay_id,assay_chembl_id,description,assay_type,predicted_category,prediction_score,predicted_bao_id,predicted_bao_term,Radioligand binding (BAO_0002776) prediction score,Binding (BAO_0002989) prediction score,...,in vivo method (BAO_0040021) prediction score,Cell phenotype (BAO_0002542) prediction score,Nucleic acid binding prediction score,Antimicrobial activity prediction score,year,document_chembl_id,bao_format,bao_label,pubmed_id,chembl_release_id
0,89,CHEMBL615209,Evaluated for its activity to inhibit rat live...,B,Protein activity,0.821,BAO_0013016,functional target-based,0.000,0.021,...,0.000,0.000,0.000,0.004,1994,CHEMBL1127895,BAO_0000019,assay format,<NA>,1
1,235,CHEMBL615267,Inhibitory activity against inosine 5'-inosine...,B,Protein activity,0.760,BAO_0013016,functional target-based,0.000,0.000,...,0.000,0.369,0.000,0.003,2003,CHEMBL1136691,BAO_0000357,single protein format,12657279,1
2,667,CHEMBL615743,5-hydroxytryptamine 1A receptor antagonism was...,F,Radioligand binding,0.749,BAO_0002776,radioligand binding assay,0.749,0.004,...,0.000,0.005,0.005,0.000,2003,CHEMBL1136114,BAO_0000019,assay format,12824042,1
3,758,CHEMBL615860,Binding affinity towards human 5-hydroxytrypta...,B,Binding,1.000,BAO_0002989,binding assay,0.001,1.000,...,0.000,0.000,0.001,0.000,2001,CHEMBL1134342,BAO_0000357,single protein format,11170639,1
4,1155,CHEMBL616100,Compound was evaluated for in vitro binding af...,B,Radioligand binding,1.000,BAO_0002776,radioligand binding assay,1.000,0.000,...,0.001,0.000,0.000,0.001,1992,CHEMBL1126190,BAO_0000221,tissue-based format,1353116,1
5,633,CHEMBL615981,Antagonistic efficacy as adenylyl cyclase assa...,F,Protein activity,0.170,BAO_0013016,functional target-based,0.002,0.073,...,0.000,0.017,0.001,0.001,1998,CHEMBL1130957,BAO_0000219,cell-based format,9873561,1
6,952,CHEMBL616131,Inhibition of [3H]- OH-DPAT binding against 5-...,B,Radioligand binding,1.000,BAO_0002776,radioligand binding assay,1.000,0.000,...,0.000,0.000,0.000,0.000,1999,CHEMBL1132537,BAO_0000357,single protein format,10602703,1
7,1288,CHEMBL616149,In vitro inhibition of [3H]spiperone binding t...,B,Radioligand binding,1.000,BAO_0002776,radioligand binding assay,1.000,0.000,...,0.000,0.000,0.000,0.000,1988,CHEMBL1124154,BAO_0000221,tissue-based format,2898533,1
8,1436,CHEMBL616309,Tested for affinity against 5-hydroxytryptamin...,B,Radioligand binding,1.000,BAO_0002776,radioligand binding assay,1.000,0.000,...,0.000,0.000,0.000,0.000,1994,CHEMBL1127438,BAO_0000019,assay format,8064801,1
9,1764,CHEMBL616550,Adenyl cyclase activity was expressed as perce...,F,Protein activity,0.510,BAO_0013016,functional target-based,0.001,0.001,...,0.000,0.001,0.009,0.004,1998,CHEMBL1131255,BAO_0000019,assay format,9632357,1
